## Testing out parenx skeletonization and voronoi approaches

Resources:
* https://github.com/nptscot/networkmerge
* https://github.com/nptscot/networkmerge
* https://github.com/anisotropi4/parenx/tree/main

In [ ]:
import glob
import os
import re
import shutil

import geopandas as gpd

from core import utils

In [8]:
# parquet is not a recognized format for parenx - convert to gpkg first

fuas = list(utils.fua_city.keys())

for fua in fuas:
    subfolder = f"../../data/{fua}/temp-parenx/"
    os.makedirs(f"../../data/{fua}/temp-parenx/", exist_ok=True)
    roads = gpd.read_parquet(f"../../data/{fua}/original/{fua}.parquet").reset_index(
        drop=True
    )
    roads.to_file(subfolder + "roads_osm.gpkg", layer="roads", engine="pyogrio")

**Now, run the bash script `parenx-run.sh` from command line**

(this implies having activated a `conda` environent in which `parenx` is installed)

`bash notebooks/methods/parenx-run.sh`

this will add to each subfolder in `temp-parenx` 2 files: voronoi.gpkg and skeletonize.gpkg. gitignoring them for now because the outputs are too large.

**reduce output file size by removing duplicated data**,  and copy to corresponding `data/{fua_id]}/parenx/` folders (in parquet format)

In [ ]:
for subfolder in glob.glob("../../data/*/temp-parenx/"):
    fua = int(re.findall(r"\d+", subfolder)[0])

    # SKELETONIZE
    os.makedirs(f"../../data/{fua}/parenx-skeletonize/", exist_ok=True)
    ske = gpd.read_file(
        filename=subfolder + "/skeletonize.gpkg", driver="fiona", layer="line"
    )
    ske.to_parquet(f"../../data/{fua}/parenx-skeletonize/{fua}.parquet")

    # VORONOI
    os.makedirs(f"../../data/{fua}/parenx-voronoi/", exist_ok=True)
    vor = gpd.read_file(
        filename=subfolder + "/voronoi.gpkg", driver="fiona", layer="line"
    )
    vor.to_parquet(f"../../data/{fua}/parenx-voronoi/{fua}.parquet")

    print(f"Done for {fua}")

Done for 869
Done for 8989
Done for 1656
Done for 4881
Done for 809
Done for 1133
Done for 4617


**Remove parenx results**

In [ ]:
for fua in fuas:
    shutil.rmtree(f"../../data/{fua}/temp-parenx/")

### Initial observations & thoughts:
* computation time: skeletonization around 10min for all 5 usecases; voronoi between 1h and 14h (salt lake city, maybe because it has the largest area, or maybe because my laptop went to sleep...)
*  it works well for some places (esp intersections, even the more complicated ones)
* major issue 1: sometimes network topology is not kept (linestrings that don't connect are merged)
* major issue 2: it creates wobbly lines